# Train your Model on Spark, score it on Data Explorer pool

<img src="https://github.com/Azure/azure-kusto-spark/raw/master/kusto_spark.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

In many use cases Machine Learning models are built and applied over data that is stored and managed by Azure Data Explorer (ADX). Most ML models are built and deployed in two steps:

* Offline training
* Real time scoring


ML Training is a long and iterative process. Commonly, a model is developed by researchers/data scientists. They fetch the training data, clean it, engineer features, try different models and tune parameters, 
repeating this cycle until the ML model meets the required accuracy and robustness. To improve accuracy, they can:

* Use a big data set, if available, that might contain hundreds of millions of records and plenty of features (dimensions and metrics)
* Train very complex models, e.g., a DNN with dozens of layers and millions of free parameters
* Perform a more exhaustive search for tuning the model’s hyper parameters
Once the model is ready, it can be deployed to production for scoring.


ML Scoring is the process of applying the model on new data to get predictions/regressions. Scoring usually needs to be done with minimal latency (near real time) for batches of streamed data.

 
Data Explorer supports running inline Python scripts that are embedded in the KQL query. The Python code runs on the existing compute nodes of Data Explorer Pools, in a distributed manner near the data. 
It can handle Data Frames containing many millions of records, partitioned and processed on multiple nodes. This optimized architecture results in great performance and minimal latency.

Specifically, for ML workloads, Data Explorer can be used for both training and scoring:

* Scoring on ADX is the ultimate solution for data that is stored on ADX, as
  * Processing is done near the data, which guarantees the fastest performance
  * Embedding the scoring Python code in KQL query is simple, robust and cheap, relative to the usage of an external scoring service that requires management, networking, security, etc.
Scoring can be done using the predict_fl() library function

* Training on ADX can be done in case the full training data set is stored in Data Explorer, the training process takes up to few minutes and doesn’t require GPUs or other special hardware
Still in many scenarios training is done on Big Data systems, such as Spark. Specifically, ML training on these systems is preferred in case that:

* The training data is not stored in ADX, but in the data lake or other external storage/db
* The training process is long (takes more than 5-10 minutes), usually done in batch/async mode

Training can be accelerated by using GPUs
Data Explorer production workflows must not be compromised by lengthy, CPU intensive, training jobs.
So we end up in a workflow that uses Spark for training, and Data Explorer for scoring. 
Training on Spark is mostly done using the Spark ML framework, that is optimized for Spark architecture, but not supported by plain vanilla Python environment like ADX Python. So how can we still score in ADX?

The solution is built from these steps:

1. Fetch the training data from Data Explorer to Spark 
1. Train an ML model in Spark
1. Convert the model to ONNX
1. Serialize and export the model to Data Explorer 
1. Score in ADX using onnxruntime

**Prerequisites:**

Enable the Python plugin on your Data Explorer Pool and load the OccupancyDetection dataset to your Data Explorer Database (see the 
[KQL script](https://ms.web.azuresynapse.net/authoring/analyze/kqlscripts/Prediction-of-Room-Occupancy?feature.useKustoClientCanary=true&feature.useKustoManagement=true&useKql=true&showKustoPool=true&workspace=%2Fsubscriptions%2F2e131dbf-96b3-4377-9c8e-de5d3047f566%2FresourceGroups%2FDEMO%2Fproviders%2FMicrosoft.Synapse%2Fworkspaces%2Fcontosoworkspace01)).

In the following example we build a logistic regression model to predict room occupancy based on Occupancy Detection data, a public dataset from UCI Repository. 
This model is a binary classifier to predict occupied or empty rooms based on temperature, humidity, light and CO2 sensors measurements. 
The example contains the full process of retrieving the data from ADX, building the model, convert it to ONNX and push it to your Data Explorer Pool. 

Finally the KQL scoring query can be run using a KQL script.

## 1. Load the data from ADX to Synapse Spark

In [ ]:
pyKusto = spark.builder.appName("kustoPySpark").getOrCreate()

In [ ]:
# Read the data from the Data Explorer table 
crp = sc._jvm.com.microsoft.azure.kusto.data.ClientRequestProperties()
crp.setOption("norequesttimeout",True)
crp.toString()

df  = spark.read \
            .format("com.microsoft.kusto.spark.synapse.datasource") \
            .option("spark.synapse.linkedService", "<DX pool linked service>") \
            .option("kustoDatabase", "<DX pool DB>") \
            .option("kustoQuery", "OccupancyDetection") \
            .option("clientRequestPropertiesJson", crp.toString()) \
            .option("readMode", 'ForceDistributedMode') \
            .load()


In [ ]:
df.take(4)

In [ ]:
df.groupBy('Test', 'Occupancy').count().show()

## 2. Train the ML model in Synapse Spark

In [ ]:
s_train_x = df.filter(df.Test == False).select('Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio')
s_train_y = df.filter(df.Test == False).select('Occupancy')
s_test_x = df.filter(df.Test == True).select('Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio')
s_test_y = df.filter(df.Test == True).select('Occupancy')

In [ ]:
df = df.withColumn('Label', df['Occupancy'].cast('int'))
df.printSchema()

In [ ]:
df.show(4)

In [ ]:
s_train = df.filter(df.Test == False)
s_test = df.filter(df.Test == True)
print(s_train.count(), s_test.count())
s_train.take(4)

In [ ]:
# Prepare the input for the model

# Spark Logistic Regression estimator requires integer label so create it from the boolean Occupancy column
df = df.withColumn('Label', df['Occupancy'].cast('int'))

# Split to train & test sets
s_train = df.filter(df.Test == False)
s_test = df.filter(df.Test == True)

# Create the Logistic Regression model
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# The Logistic Regression estimator expects the features in a single column so create it using VectorAssembler
features = ('Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio')
assembler = VectorAssembler(inputCols=features,outputCol='Features')
s_train_features = assembler.transform(s_train)
s_train_features.take(4)
lr = LogisticRegression(labelCol='Label', featuresCol='Features',maxIter=10)
s_clf = lr.fit(s_train_features)

# Predict the training set
s_predict_train = s_clf.transform(s_train_features)

# Predict the testing set
s_test_features = assembler.transform(s_test)
s_predict_test = s_clf.transform(s_test_features)
s_predict_test.select(['Timestamp', 'Features', 'Label', 'prediction']).show(10)


In [ ]:
s_test_features = assembler.transform(s_test)
s_test_features.take(4)
s_predict_test = s_clf.transform(s_test_features)
s_predict_test.show(10)

In [ ]:
# Calculate accuracy on the testing set

import pyspark.sql.functions as F
check = s_predict_test.withColumn('correct', F.when(F.col('Label') == F.col('prediction'), 1).otherwise(0))
check.groupby('correct').count().show()
accuracy = check.filter(check['correct'] == 1).count()/check.count()*100

 ## 3. Convert the model to ONNX

In [ ]:
from onnxmltools import convert_sparkml
from onnxmltools.convert.sparkml.utils import FloatTensorType

initial_types = [('Features', FloatTensorType([None, 5]))]
onnx_model = convert_sparkml(s_clf, 'Occupancy detection Pyspark Logistic Regression model', initial_types, spark_session = pyKusto)
onnx_model

## 4. Export the model to ADX

In [ ]:
import datetime
import pandas as pd

smodel = onnx_model.SerializeToString().hex()
models_tbl = 'ML_Models'
model_name = 'Occupancy_Detection_LR'

# Create a DataFrame containing a single row with model name, training time and
# the serialized model, to be appended to the models table
now = datetime.datetime.now()
dfm = pd.DataFrame({'name':[model_name], 'timestamp':[now], 'model':[smodel]})
sdfm = spark.createDataFrame(dfm)
sdfm.show()

In [ ]:
extentsCreationTime = sc._jvm.org.joda.time.DateTime.now()

sp = sc._jvm.com.microsoft.kusto.spark.datasink.SparkIngestionProperties(
        True, None, None, None, None, extentsCreationTime, None, None)

In [ ]:
# Write the model to Data Explorer
sdfm.write.format("com.microsoft.kusto.spark.synapse.datasource") \
.option("spark.synapse.linkedService", "<DX pool linked service>") \
.option("kustoDatabase", "<DX pool DB>") \
.option("kustoTable", models_tbl) \
.option("sparkIngestionPropertiesJson", sp.toString()) \
.option("tableCreateOptions","CreateIfNotExist") \
.mode("Append") \
.save()

## 5. Score in ADX
Is done by calling predict_onnx_fl() You can either install this function in your database, or call it in ad-hoc manner:


In [ ]:
prediction_query = '''
let predict_onnx_fl=(samples:(*), models_tbl:(name:string, timestamp:datetime, model:string), model_name:string, features_cols:dynamic, pred_col:string)
{

    let model_str = toscalar(models_tbl | where name == model_name | top 1 by timestamp desc | project model);
    let kwargs = pack('smodel', model_str, 'features_cols', features_cols, 'pred_col', pred_col);
    let code = ```if 1:
    import binascii
    smodel = kargs["smodel"]
    features_cols = kargs["features_cols"]
    pred_col = kargs["pred_col"]
    bmodel = binascii.unhexlify(smodel)
    features_cols = kargs["features_cols"]
    pred_col = kargs["pred_col"]
    import onnxruntime as rt
    sess = rt.InferenceSession(bmodel)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    df1 = df[features_cols]
    predictions = sess.run([label_name], {input_name: df1.values.astype(np.float32)})[0]
    result = df
    result[pred_col] = pd.DataFrame(predictions, columns=[pred_col])
    ```;
    samples | evaluate python(typeof(*), code, kwargs)
};
OccupancyDetection 
| where Test == 1
| extend pred_Occupancy=int(null)
| invoke predict_onnx_fl(ML_Models, 'Occupancy_Detection_LR', pack_array('Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio'), 'pred_Occupancy')
| summarize correct = countif(Occupancy == pred_Occupancy), incorrect = countif(Occupancy != pred_Occupancy), total = count()
| extend accuracy = 100.0*correct/total
'''

In [ ]:
spdf  = spark.read \
            .format("com.microsoft.kusto.spark.synapse.datasource") \
            .option("spark.synapse.linkedService", "<DX pool linked service>") \
            .option("kustoDatabase", "<DX pool DB>") \
            .option("kustoQuery", prediction_query) \
            .option("clientRequestPropertiesJson", crp.toString()) \
            .option("readMode", 'ForceDistributedMode') \
            .load()
spdf.show ()